# Resonance 1 fr and 1/Qi vs T

In [ ]:
# Example usage
from iminuit.cost import LeastSquares
from FitAPI import Fitter, Model
import numpy as np
import numpy as np
import sys
sys.path.append('../Graphics')
from Graphs import fitPlotter
import matplotlib.pyplot as plt

def S21_model(f: np.ndarray, f0: float, phi: float, Qt: float, Qc: float, A: float, B: float, C: float, D: float, K: float) -> np.ndarray:
    return (A + B*(f-fmin) + C*(f-fmin)**2 + D*(f-fmin)**3) + K * np.abs((1 - (Qt/np.abs(Qc))*np.exp(1j*phi)/(1 + 2j*Qt*((f-fmin) - f0)/fmin)))

initial_guess = {
    "f0": 0.01,
    "phi": 0.01,
    "Qc": 1e3,
    "A": 1.0,
    "B": 1e-9,
    "C": 1e-18,
    "D": 1e-27
}

# Estimate the initial guess of Qt
def peak_width(datax, datay):
    half_height_value = np.min(datay) + (np.max(datay) - np.min(datay)) / np.sqrt(2)
    hits = []
    above = datay[0] > half_height_value
    for i in range(1, len(datay)):
        new_above = datay[i] > half_height_value
        if new_above != above: 
            hits.append((datax[i] + datax[i-1]) / 2)
            above = new_above
    return abs(hits[-1] - hits[0])

result_fr = []
err_fr = []
result_Qi = []
err_Qi = []
T = []

f = []
y = []
for i in range(40, 116, 2):
    path = '../Data/fit_test/gap run12/Q_res' +str(i) +'.txt'
    data = np.loadtxt(path, delimiter=',', skiprows=0)
    f.append(data[:,0])
    I = data[:,1]
    Q = data[:,2]
    # Convert to module
    y.append(np.sqrt(I**2 + Q**2))
    T.append(410 - 10*((i-40)/2))
    
print(f[0])

[3.174000e+09 3.174003e+09 3.174006e+09 ... 3.178494e+09 3.178497e+09
 3.178500e+09]


In [10]:


#(116 - 40)/2 = 38
#(410-30)/10 = 38

for i in range(40, 116, 2):
    f_i = f[int((i-40)/2)]
    y_i = y[int((i-40)/2)]
    
    fmin = f_i[np.argmin(y_i)]
    
    # Estimate the initial guess of Qt
    #Q_i = fmin / peak_width(f, -y)
    #Qt_guess = 1/(1/initial_guess['Qc'] + 1/Q_i)
    Qt_guess = fmin / peak_width(f_i, -y_i)
    
    print(f"Qt_guess: {Qt_guess}")
    initial_guess['Qt'] = Qt_guess

    # Estimate the initial guess of K
    initial_guess['K'] = (np.max(y_i) - np.min(y_i)) * initial_guess['Qc'] / initial_guess['Qt']
    print(f"Initial guess: {initial_guess}")

    fit_data = np.column_stack((f_i, y_i))
    fitter = Fitter(model_function=S21_model, 
                    param_names=["f0", "phi", "Qt", "Qc", "A", "B", "C", "D", "K"], 
                    data=fit_data, 
                    loss_function=LeastSquares,
                    params_initial_guess=initial_guess)

    result = fitter.fit_quasi_magicus()
    result_dict = result.values.to_dict()
    err_dict = result.errors.to_dict()
    
    result_fr.append(result_dict['f0'] + fmin)
    err_fr.append(err_dict['f0'])
    
    result_Qi.append(abs((1/result_dict['Qt']-1/result_dict['Qc'])**-1))
    Qt = result_dict['Qt']
    Qc = result_dict['Qc']
    err_Qi.append(np.sqrt((err_dict['Qt'] * Qt**-2)**2 + (err_dict['Qc'] * Qc**-2)**2) * ((1/Qt-1/Qc)**-2))
    
    
print(result_fr)
print(result_Qi)
print(T) 

plt.errorbar(T, result_fr, err_fr, linestyle='-', label='fr')
plt.ylabel('Resonance frequency (GHz)')
plt.xlabel('Temperature (mK)')
plt.show()

plt.errorbar(T, 1/result_Qi, err_Qi*result_Qi**-2, linestyle='-', label='Qi')
plt.ylabel('1/Internal quality factor')
plt.xlabel('Temperature (mK)')
plt.show()


Qt_guess: 27629.41739130435
Initial guess: {'f0': 0.01, 'phi': 0.01, 'Qc': 1000.0, 'A': 1.0, 'B': 1e-09, 'C': 1e-18, 'D': 1e-27, 'Qt': np.float64(27629.41739130435), 'K': np.float64(0.0024407327439281436)}
First total fit failed
Partial fit 0
Partial fit 0 converged bad , p-value = 0.0
Partial fit 1
Partial fit 1 converged bad , p-value = 0.0
Partial fit 2
Partial fit 2 converged bad , p-value = 0.0
Partial fit 3
Partial fit 3 converged bad , p-value = 0.0
Partial fit 4
Partial fit 4 converged bad , p-value = 0.0
Partial fit 5
Partial fit 5 converged bad , p-value = 0.0
Partial fit 6
Partial fit 6 converged bad , p-value = 0.0
Partial fit 7
Partial fit 7 converged bad , p-value = 0.0
Partial fit 8
Partial fit 8 converged bad , p-value = 0.0
Partial fit 9
Partial fit 9 converged bad , p-value = 0.0


RuntimeError: The fit did not converge. All partial fits failed